# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [4]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [6]:
# 1.1 Idade média das mães e dos pais por município
df = sinasc_raw.copy()                       # garante o DataFrame correto
df['IDADEMAE'] = pd.to_numeric(df['IDADEMAE'], errors='coerce')

# Ajuste: nome da coluna do pai. Se não existir, criaremos uma coluna vazia
if 'IDADEPAI' not in df.columns:
    df['IDADEPAI'] = pd.Series(dtype='float')

df['IDADEPAI'] = pd.to_numeric(df['IDADEPAI'], errors='coerce')

idade_media = (
    df.groupby('munResNome')
      .agg(idade_media_mae=('IDADEMAE','mean'),
           idade_media_pai=('IDADEPAI','mean'))
      .sort_values('idade_media_mae', ascending=False)
      .round(2)
      .reset_index()
)

idade_media.head()

,munResNome,idade_media_mae,idade_media_pai
0,Ministro Andreazza,27.90,33.85
1,Colorado do Oeste,27.60,34.35
2,Castanheiras,27.28,30.39
3,Cerejeiras,27.24,31.65
4,Primavera de Rondônia,27.02,31.89


## Insights
    Nos dados apresentados por municípios a  idade média tanto das mães como dos pais não apresenta uma grande variação, mas comparando mães e pais, estem tendem a ser em média alguns anos mais velhos.


### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [7]:
# 1.2 Peso médio por sexo para nascimentos numa data específica por faixa de escolaridade da mãe
df = sinasc_raw.copy()
df['DTNASC'] = pd.to_datetime(df['DTNASC'], errors='coerce')
df['PESO'] = pd.to_numeric(df['PESO'], errors='coerce')
df['SEXO'] = df['SEXO'].astype(str)
df['ESCMAE'] = df['ESCMAE'].astype(str)

aniversario = '2019-01-10'                  # altere para sua data
mask = df['DTNASC'] == pd.to_datetime(aniversario)
df_aniv = df.loc[mask].dropna(subset=['PESO','SEXO','ESCMAE'])

pv = (
    df_aniv.groupby(['ESCMAE','SEXO'])['PESO']
          .mean()
          .reset_index()
          .rename(columns={'PESO':'peso_medio'})
          .sort_values(['ESCMAE','SEXO'])
)

pv

,ESCMAE,SEXO,peso_medio
0,1 a 3 anos,Feminino,3651.000000
1,1 a 3 anos,Masculino,3078.333333
2,12 anos ou mais,Feminino,3351.111111
3,12 anos ou mais,Masculino,3015.428571
4,4 a 7 anos,Feminino,3152.375000
5,4 a 7 anos,Masculino,3105.000000
6,8 a 11 anos,Feminino,3120.909091
7,8 a 11 anos,Masculino,3497.550000
8,nan,Feminino,3825.000000
9,nan,Masculino,4470.000000


 ## Insights:
 
    • Padrão geral: há variação de peso médio entre faixas de escolaridade da mãe e entre sexos — nem sempre os meninos têm peso maior; em algumas faixas as meninas aparecem com média superior.

    • Valores inesperados: as linhas com ESCMAE = nan mostram médias muito altas (Feminino 3825, Masculino 4470) — isso pode indicar poucas observações ou erros de digitação/inserção.

    • Heterogeneidade entre faixas: por exemplo, em “1 a 3 anos” a média feminina (3651) é bem maior que a masculina (3078), enquanto em “8 a 11 anos” o masculino (3497) supera o feminino (3103). Isso sugere que a escolaridade da mãe pode estar associada a diferenças nos pesos ao nascer, mas a direção não é uniforme.


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [8]:
# 1.3 Município com menos nascimentos em 2019 e estatísticas de idade
df = sinasc_raw.copy()
df['DTNASC'] = pd.to_datetime(df['DTNASC'], errors='coerce')
df['IDADEMAE'] = pd.to_numeric(df['IDADEMAE'], errors='coerce')
if 'IDADEPAI' not in df.columns:
    df['IDADEPAI'] = pd.Series(dtype='float')
df['IDADEPAI'] = pd.to_numeric(df['IDADEPAI'], errors='coerce')

df2019 = df[df['DTNASC'].dt.year == 2019].dropna(subset=['munResNome'])

contagem_mun = (
    df2019.groupby('munResNome')
          .size()
          .reset_index(name='n_nascimentos')
          .sort_values('n_nascimentos', ascending=True)
)

contagem_mun.head()    # mostra os municípios com menos nascimentos (os primeiros)

,munResNome,n_nascimentos
28,Município ignorado - RO,1
11,Castanheiras,32
36,Pimenteiras do Oeste,40
39,Primavera de Rondônia,43
34,Parecis,44


In [12]:
# 1.3 — Municípios com menos nascimentos em 2019 e estatísticas de idade
import numpy as np

df = sinasc_raw.copy()
df['DTNASC'] = pd.to_datetime(df['DTNASC'], errors='coerce')
df['IDADEMAE'] = pd.to_numeric(df['IDADEMAE'], errors='coerce')
if 'IDADEPAI' not in df.columns:
    df['IDADEPAI'] = pd.Series(dtype='float')
df['IDADEPAI'] = pd.to_numeric(df['IDADEPAI'], errors='coerce')

df2019 = df[df['DTNASC'].dt.year == 2019].dropna(subset=['munResNome'])

contagem_mun = (
    df2019.groupby('munResNome')
          .size()
          .reset_index(name='n_nascimentos')
          .sort_values('n_nascimentos', ascending=True)
)

# todos os municípios que têm a contagem mínima
min_count = contagem_mun['n_nascimentos'].min()
mun_minimos = contagem_mun[contagem_mun['n_nascimentos'] == min_count]['munResNome'].tolist()

def estatisticas_idade(subdf, col):
    return {
        'n_registros': int(subdf[col].notna().sum()),
        'idade_media': round(float(subdf[col].mean()),2) if subdf[col].notna().any() else np.nan,
        'idade_minima': float(subdf[col].min()) if subdf[col].notna().any() else np.nan,
        'idade_maxima': float(subdf[col].max()) if subdf[col].notna().any() else np.nan
    }

rows = []
for mun in mun_minimos:
    sub = df2019[df2019['munResNome'] == mun]
    mae = estatisticas_idade(sub, 'IDADEMAE')
    pai = estatisticas_idade(sub, 'IDADEPAI')
    rows.append({
        'munResNome': mun,
        'n_nascimentos': int(sub.shape[0]),
        'mae_n_reg': mae['n_registros'],
        'mae_idade_media': mae['idade_media'],
        'mae_idade_min': mae['idade_minima'],
        'mae_idade_max': mae['idade_maxima'],
        'pai_n_reg': pai['n_registros'],
        'pai_idade_media': pai['idade_media'],
        'pai_idade_min': pai['idade_minima'],
        'pai_idade_max': pai['idade_maxima'],
    })

df_mun_min = pd.DataFrame(rows).sort_values('n_nascimentos')
display(contagem_mun.head(30))   # mostra os municípios com menos nascimentos
display(df_mun_min)             # estatísticas detalhadas para os municípios com contagem mínima

# Opcional: ver os N municípios com menos nascimentos (ajuste N)
N = 10
bottomN = contagem_mun.head(N)['munResNome'].tolist()
rowsN = []
for mun in bottomN:
    sub = df2019[df2019['munResNome'] == mun]
    mae = estatisticas_idade(sub, 'IDADEMAE')
    pai = estatisticas_idade(sub, 'IDADEPAI')
    rowsN.append({
        'munResNome': mun,
        'n_nascimentos': int(sub.shape[0]),
        'mae_idade_media': mae['idade_media'],
        'mae_idade_min': mae['idade_minima'],
        'mae_idade_max': mae['idade_maxima'],
        'pai_idade_media': pai['idade_media'],
        'pai_idade_min': pai['idade_minima'],
        'pai_idade_max': pai['idade_maxima'],
    })

df_bottomN = pd.DataFrame(rowsN).sort_values('n_nascimentos')
display(df_bottomN)

,munResNome,n_nascimentos
28,Município ignorado - RO,1
11,Castanheiras,32
36,Pimenteiras do Oeste,40
39,Primavera de Rondônia,43
34,Parecis,44
40,Rio Crespo,50
44,São Felipe D'Oeste,54
47,Teixeirópolis,64
7,Cacaulândia,75
6,Cabixi,80


,munResNome,n_nascimentos,mae_n_reg,mae_idade_media,mae_idade_min,mae_idade_max,pai_n_reg,pai_idade_media,pai_idade_min,pai_idade_max
0,Município ignorado - RO,1,1,24.0,24.0,24.0,1,22.0,22.0,22.0


,munResNome,n_nascimentos,mae_idade_media,mae_idade_min,mae_idade_max,pai_idade_media,pai_idade_min,pai_idade_max
0,Município ignorado - RO,1,24.00,24.0,24.0,22.00,22.0,22.0
1,Castanheiras,32,27.28,17.0,39.0,30.39,17.0,43.0
2,Pimenteiras do Oeste,40,25.07,14.0,40.0,35.60,25.0,45.0
3,Primavera de Rondônia,43,27.02,16.0,39.0,31.89,21.0,44.0
4,Parecis,44,26.45,16.0,41.0,32.47,18.0,61.0
5,Rio Crespo,50,26.66,16.0,39.0,28.00,28.0,28.0
6,São Felipe D'Oeste,54,26.48,17.0,41.0,31.40,18.0,57.0
7,Teixeirópolis,64,26.03,16.0,39.0,31.50,20.0,67.0
8,Cacaulândia,75,25.55,16.0,42.0,36.33,28.0,50.0
9,Cabixi,80,26.04,13.0,39.0,34.33,19.0,47.0


 ## Insights:
    • Pais consistentemente mais velhos que mães — em quase todos os municípios a média do pai é ~3–10 anos maior que a da mãe, padrão demográfico esperado, mas a magnitude varia muito entre municípios (ex.: Cacaulândia pai média 36.33 vs mãe 25.55).
    • Faixa etária das mães inclui idades muito jovens — há mães com 13–17 anos em alguns municípios; isso indica necessidade de atenção em políticas públicas e potenciais diferenças socioeconômicas locais.
    • Municípios com maiores diferenças pai‑mãe — locais como Cacaulândia e Pimenteiras do Oeste mostram diferença média elevada; pode apontar fatores culturais ou de seleção (uniões com grande diferença etária) que merecem investigação.



### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [14]:
# 1) preparar cópia e tipos
df = sinasc_raw.copy()
df['DTNASC'] = pd.to_datetime(df['DTNASC'], errors='coerce')
df['QTDFILVIVO'] = pd.to_numeric(df['QTDFILVIVO'], errors='coerce')
df['IDADEMAE'] = pd.to_numeric(df['IDADEMAE'], errors='coerce')

# garantir coluna IDADEPAI caso não exista
if 'IDADEPAI' not in df.columns:
    df['IDADEPAI'] = pd.Series(dtype='float')
df['IDADEPAI'] = pd.to_numeric(df['IDADEPAI'], errors='coerce')

# 2) filtrar março de 2019
mask_marco = (df['DTNASC'].dt.year == 2019) & (df['DTNASC'].dt.month == 3)
df_mar = df.loc[mask_marco].dropna(subset=['munResNome'])   # garantir município válido

# 3) contar nascimentos por município em março
contagem_mar = df_mar.groupby('munResNome').size().reset_index(name='n_nascimentos').sort_values('n_nascimentos', ascending=False)

# mostrar o ranking (opcional)
display(contagem_mar.head(20))

# 4) município com mais nascimentos em março
if contagem_mar.shape[0] == 0:
    print("Não há registros em março de 2019 no dataset.")
else:
    top_mun = contagem_mar.iloc[0]['munResNome']
    top_n = int(contagem_mar.iloc[0]['n_nascimentos'])
    print(f"Município com mais nascimentos em março/2019: {top_mun}  (nascimentos = {top_n})")

    # 5) filtrar registros desse município
    sub = df_mar[df_mar['munResNome'] == top_mun]

    # 6) estatísticas de QTDFILVIVO
    qtv = sub['QTDFILVIVO']
    qtv_stats = {
        'qtv_n_registros': int(qtv.notna().sum()),
        'qtv_media': float(qtv.mean()) if qtv.notna().any() else np.nan,
        'qtv_min': float(qtv.min()) if qtv.notna().any() else np.nan,
        'qtv_max': float(qtv.max()) if qtv.notna().any() else np.nan
    }

    # 7) estatísticas de idade das mães
    mae = sub['IDADEMAE']
    mae_stats = {
        'mae_n_registros': int(mae.notna().sum()),
        'mae_idade_media': float(mae.mean()) if mae.notna().any() else np.nan,
        'mae_idade_min': float(mae.min()) if mae.notna().any() else np.nan,
        'mae_idade_max': float(mae.max()) if mae.notna().any() else np.nan
    }

    # 8) estatísticas de idade dos pais
    pai = sub['IDADEPAI']
    pai_stats = {
        'pai_n_registros': int(pai.notna().sum()),
        'pai_idade_media': float(pai.mean()) if pai.notna().any() else np.nan,
        'pai_idade_min': float(pai.min()) if pai.notna().any() else np.nan,
        'pai_idade_max': float(pai.max()) if pai.notna().any() else np.nan
    }

    # 9) mostrar resultados arredondados quando aplicável
    from pprint import pprint
    print("\nEstatísticas QTDFILVIVO (filhos vivos) no município:")
    pprint({k: (round(v,2) if isinstance(v,(float)) and not np.isnan(v) else v) for k,v in qtv_stats.items()})
    print("\nEstatísticas idade das MÃES no município:")
    pprint({k: (round(v,2) if isinstance(v,(float)) and not np.isnan(v) else v) for k,v in mae_stats.items()})
    print("\nEstatísticas idade dos PAIS no município:")
    pprint({k: (round(v,2) if isinstance(v,(float)) and not np.isnan(v) else v) for k,v in pai_stats.items()})

,munResNome,n_nascimentos
36,Porto Velho,744
23,Ji-Paraná,188
51,Vilhena,148
4,Ariquemes,141
8,Cacoal,133
22,Jaru,80
20,Guajará-Mirim,75
40,Rolim de Moura,69
32,Ouro Preto do Oeste,67
34,Pimenta Bueno,62


Município com mais nascimentos em março/2019: Porto Velho  (nascimentos = 744)

Estatísticas QTDFILVIVO (filhos vivos) no município:
{'qtv_max': 9.0, 'qtv_media': 1.3, 'qtv_min': 0.0, 'qtv_n_registros': 627}

Estatísticas idade das MÃES no município:
{'mae_idade_max': 44.0,
 'mae_idade_media': 26.6,
 'mae_idade_min': 14.0,
 'mae_n_registros': 744}

Estatísticas idade dos PAIS no município:
{'pai_idade_max': 62.0,
 'pai_idade_media': 34.63,
 'pai_idade_min': 19.0,
 'pai_n_registros': 62}


In [23]:
# Top6 com estatísticas de QTDFILVIVO, IDADEMAE e IDADEPAI
import pandas as pd
import numpy as np

df = sinasc_raw.copy()
df['DTNASC'] = pd.to_datetime(df['DTNASC'], errors='coerce')
df['QTDFILVIVO'] = pd.to_numeric(df['QTDFILVIVO'], errors='coerce')
df['IDADEMAE'] = pd.to_numeric(df['IDADEMAE'], errors='coerce')
if 'IDADEPAI' not in df.columns:
    df['IDADEPAI'] = pd.Series(dtype='float')
df['IDADEPAI'] = pd.to_numeric(df['IDADEPAI'], errors='coerce')

# filtrar março/2019
mask_mar = (df['DTNASC'].dt.year == 2019) & (df['DTNASC'].dt.month == 3)
df_mar = df.loc[mask_mar].dropna(subset=['munResNome']).copy()

# top 6 municípios por nascimentos em março
contagem = df_mar.groupby('munResNome').size().reset_index(name='n_nascimentos')
top6 = contagem.sort_values('n_nascimentos', ascending=False).head(6)['munResNome'].tolist()

def stats(s):
    s = s.dropna()
    return {
        'n': int(s.count()),
        'mean': float(s.mean()) if not s.empty else np.nan,
        'min': float(s.min()) if not s.empty else np.nan,
        'max': float(s.max()) if not s.empty else np.nan
    }

rows = []
for mun in top6:
    sub = df_mar[df_mar['munResNome'] == mun]
    q = stats(sub['QTDFILVIVO'])
    mae = stats(sub['IDADEMAE'])
    pai = stats(sub['IDADEPAI'])
    rows.append({
        'munResNome': mun,
        'n_nascimentos': int(sub.shape[0]),
        # QTDFILVIVO
        'qtv_n': q['n'],
        'qtv_mean': round(q['mean'],2) if not np.isnan(q['mean']) else np.nan,
        'qtv_min': q['min'],
        'qtv_max': q['max'],
        # mãe
        'mae_n': mae['n'],
        'mae_idade_mean': round(mae['mean'],2) if not np.isnan(mae['mean']) else np.nan,
        'mae_idade_min': mae['min'],
        'mae_idade_max': mae['max'],
        # pai
        'pai_n': pai['n'],
        'pai_idade_mean': round(pai['mean'],2) if not np.isnan(pai['mean']) else np.nan,
        'pai_idade_min': pai['min'],
        'pai_idade_max': pai['max'],
    })

df_top6_stats = pd.DataFrame(rows).sort_values('n_nascimentos', ascending=False).reset_index(drop=True)
df_top6_stats

,munResNome,n_nascimentos,qtv_n,qtv_mean,qtv_min,qtv_max,mae_n,mae_idade_mean,mae_idade_min,mae_idade_max,pai_n,pai_idade_mean,pai_idade_min,pai_idade_max
0,Porto Velho,744,627,1.30,0.0,9.0,744,26.60,14.0,44.0,62,34.63,19.0,62.0
1,Ji-Paraná,188,187,0.83,0.0,4.0,188,25.38,13.0,44.0,95,30.83,16.0,56.0
2,Vilhena,148,147,0.98,0.0,5.0,148,26.41,16.0,39.0,16,33.56,27.0,47.0
3,Ariquemes,141,139,1.06,0.0,4.0,141,25.30,14.0,42.0,3,28.67,28.0,29.0
4,Cacoal,133,132,0.97,0.0,7.0,133,26.47,15.0,40.0,122,30.01,17.0,47.0
5,Jaru,80,78,1.03,0.0,5.0,80,26.55,14.0,40.0,10,33.50,21.0,41.0


## Insight
    -Porto Velho teve 744 nascimentos em março — muito acima do segundo município (Ji‑Paraná, 188).
    
    - Perfil reprodutivo (filhos vivos), média QTDFILVIVO ≈ 1.3; mínimo 0 e máximo 9 — a maioria tem 0–2 filhos vivos, mas existem famílias com alta paridade (até 9), que podem demandar atenção social/saúde.
    
     - Número de registros usados em QTDFILVIVO = 627, menor que o total de nascimentos (744), indicando faltantes nessa variável.
     
    - Idade das mães idade média ≈ 26.6 anos; intervalo observado 14 a 44. 
    
    - Presença de mães muito jovens (idade mínima 14) — sinal relevante para políticas de saúde reprodutiva e educação sexual.
    
    - Idade dos pais, em média ~8 anos mais velhos que as mães neste município.
    
    - Número de registros dos pais = 627 (faltantes importantes), então estatísticas sobre pais podem estar enviesadas, cobertura completa para mães (mae_n_registros = 744)
    
## Qualidade e confiabilidade dos dados
    - Diferença no número de registros entre mães (744) e pais/qtdeFilVivo (627) mostra dados incompletos em algumas variáveis; seria preciso cuidar disso antes de inferir causalidade.
    
    - Valores extremos (mães com 14 anos; pais com 62; paridade = 9) merecem checagem de outliers/erros de entrada.


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.